# 第10节 显示有限差分法计算期权价格
* [10.1 简介](#10_1)
* [10.2 计算美式看跌期权价格算法](#10_2)
* [10.3 算法Python代码实现](#10_3)
* [10.4 参考资料](#10_4)

## <a name="10_1"><a/> 10.1 简介
* $\bf有限差分法$  
    &nbsp; &nbsp; &nbsp; &nbsp; 将微分方程表示为定义在离散的格点上的差分方程，由给定的边界条件通过相近格点间的差分关系迭代计算出未知边界上的数值。显式和隐式差分法的区别主要在于替代计算微分方程的差分方程中对微分项的离散化近似表示不同（主要是$\frac{\partial f}{\partial t}$项的表示不同）。使得显示差分法可以直接由已知边界条件递推出目标边界上的数值，而隐式差分法中则不断需要解方程组进行递推计算。但显示差分法计算出的结果不一定稳定，小的误差可能被放大使得计算出的结果不对。  
* $\bf显式差分法计算期权价格$  
    &nbsp; &nbsp; &nbsp; &nbsp; 我们知道期权价格满足微分方程：
    $$\frac{\partial f}{\partial t} + rS\frac{\partial f}{\partial S} +\frac{1}{2}\sigma^2\frac{\partial^2f}{\partial S^2} = rf \; .$$
将其写成差分形式：
    $$rf(i-1,j) -\frac{f(i, j)-f(i-1,j)}{\Delta t} = rS(i,j)\frac{f(i,j+1)-f(i,j-1)}{2\Delta S}+\frac{1}{2}\sigma^2S^2(i,j)\frac{f(i,j+1)+f(i,j-1)-2f(i,j)}{\Delta S^2} \;.$$
其中$f(i,j) = f(i\Delta t, j\Delta S),\;S(i,j)=j\Delta S,\; \Delta t =T/N, \; \Delta S=S_{max}/M$。  
然后我们可以将$i-1$时刻的$f(i-1,j)$用$i$时刻的数值表示：
    $$f(i-1,j) = a_jf(i,j-1)+b_jf(i,j)+c_jf(i,j-1) \;.$$
    $$a_j = \frac{1}{1+r\Delta t}\left(-\frac{1}{2}j\Delta t+\frac{1}{2}\sigma^2j^2\Delta t\right), \;\;b_j = \frac{1}{1+r\Delta t}(1-\sigma^2j^2\Delta t),$$
    $$c_j = \frac{1}{1+r\Delta t}\left(\frac{1}{2}j\Delta t+\frac{1}{2}\sigma^2j^2\Delta t\right) \;.$$
    <br/>&nbsp; &nbsp; &nbsp; &nbsp; 这样我们就可以从期权执行时刻开始一列一列往回递推期权价格。需要说明的是，对于股价的上下边界$S_{max}$和0，如果我们考虑欧式看跌期权，则$S_{max}$只要足够大（比如$3S_0$），则上边界处期权价格都可以近似为0，下边界$S=0$处期权价格都为执行价格$K$。欧式看涨期权的价格可以由看跌-看涨平价关系间接得出。对于美式看涨期权，由于其总是不会被提前执行，所以其价格等于欧式看涨，对于美式看跌期权，则只需要在价格递推时额外考虑应不应该在当前格点行使期权。

## <a name="10_2"><a/> 10.2 计算美式看跌期权价格算法  
1. 确定时间和股价变化范围，$0\leq t\leq T,\; 0\leq S\leq S_{max},\;S_{max}=3S_0$，并将区间离散化为二维等间距网格点，$\Delta t = T/N, \; \Delta S = S_{max}/M$。
1. 确定三个边界上的期权价格，$t=T$时$f(N,j) = \max{(0, K-j\Delta S)}$，$S=0$时$f(i,0) = K$，$S=S_{max}$时$f(i,M)=0$。
1. 从$t=T$的边界开始，往回递推出上一个时刻所有格点上的期权价格。递推公式使用上面简介中给出的表达式。
1. 考虑应不应该在新计算出的格点处执行美式看跌期权，即更新$f(i,j) = \max{(f(i,j), K-\Delta Sj)}$。
1. 重复步骤3、4直到初始时刻，然后使用线性插值法得出$S=S_0$处的期权价格。

## <a name="10_3"><a/> 10.3 算法Python代码实现

In [9]:
def American_put_explicit(r, sigma, S_0, K, T, M, N):
    # S_max = 3*S_0, 分为 M 段，T分为 N 段。
    dS = 3*S_0/M
    dt = T/N
    
    # 使用两列格点期权价格不断往回迭代计算, f1为左边列，f2为右边列。
    f1 = [max(0.0, K-i*dS) for i in range(M+1)]
    f2 = list(f1)

    for i in range(N-1, -1, -1):
        f2 = list(f1)
        # 上下界边界条件。
        f1[0] = K
        f1[-1] = 0
        for j in range(1, M, 1):
            """ M, N 的选取应该最好使得 sigma*sigma*j*j*dt 小于 1。
            """
            f1[j] = f2[j-1]*(-0.5*r*j*dt+0.5*sigma*sigma*j*j*dt)/(1+r*dt)
            f1[j] += f2[j]*(1-sigma*sigma*j*j*dt)/(1+r*dt)
            f1[j] += f2[j+1]*(0.5*r*j*dt+0.5*sigma*sigma*j*j*dt)/(1+r*dt)
        # 判断是否应该在格点执行美式期权。
        for j in range(M+1):
            f1[j] = max(f1[j], K-j*dS)
    
    # 线性插值得期权初始时刻在 S_0 处的价格。
    pos = int(S_0/dS)
    put_price = f1[pos] + (f1[pos+1]-f1[pos])/dS*(S_0-dS*pos)

    return put_price


# 计算示例。
if __name__ == "__main__":
    put_price = American_put_explicit(0.1, 0.4, 50, 60, 1.0, 150, 50000)
    print("American put price: {0:0.5f}".format(put_price))

American put price: 12.07188


## <a name="10.4"><a/> 10.4 参考资料 
1. 《期权、期货及其他衍生产品》，John C. Hull 著，王勇、索吾林译。